# Q1. Install MLflow
What version of MLFlow do you have?

In [1]:
! mlflow --version

mlflow, version 1.17.0


# Q2. Download and preprocess the data
We'll use the Green Taxi Trip Records dataset.

Download the data for January, February and March 2021 in parquet format from here.

Use the script preprocess_data.py located in the folder homework to preprocess the data.

The script will

load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),
fit a DictVectorizer on the training set (January 2021 data),
save the preprocessed datasets and the model to disk
Your task is to download the datasets and then execute this command:

python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
Tip: go to 02-experiment-tracking/homework/ folder before executing the command and change the value of <TAXI_DATA_FOLDER> to the location where you saved the data.

How many files were saved to OUTPUT_FOLDER?

1
3
4
7


In [21]:
! python3 preprocess_data.py --raw_data_path '/home/elite/Documents/DataTalksClub/MLOpsZoomcamp_HW/data' --dest_path ./output

In [25]:
! ls /home/elite/Documents/DataTalksClub/MLOpsZoomcamp_HW/data

dv.pkl				green_tripdata_2021-03.parquet	valid.pkl
green_tripdata_2021-01.parquet	test.pkl
green_tripdata_2021-02.parquet	train.pkl


There are 4 new files

# Q3. Train a model with autolog
We will train a random forest regressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script train.py for this exercise, which can be also found in the folder homework. The script will load the datasets produced by the previous step, train the model on the training set and finally calculate the RMSE on the validation set.

Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a with mlflow.start_run(): statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

How many parameters are automatically logged by MLflow?

19
17
10
20

In [27]:
! python3 train.py --data_path ./data

2022/05/31 07:13:45 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/05/31 07:13:45 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '189194cb189b4e34a18384c2e4021735', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


In [28]:
! ls /home/elite/Documents/DataTalksClub/MLOpsZoomcamp_HW/mlruns/0/189194cb189b4e34a18384c2e4021735/params

bootstrap     max_leaf_nodes	     min_samples_split	       random_state
ccp_alpha     max_samples	     min_weight_fraction_leaf  verbose
criterion     min_impurity_decrease  n_estimators	       warm_start
max_depth     min_impurity_split     n_jobs
max_features  min_samples_leaf	     oob_score


This shows 18 params. The options do not have 18. So, I will select 17, considering verbose is not really a parameter.

# Q4. Launch the tracking server locally
We want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to

launch the tracking server on your local machine
select a SQLite db for the backend store and a folder called artifacts for the artifacts store
You should keep the tracking server running to work on the next two exercises that use the server.

In addition to backend-store-uri, what else do you need to pass to properly configure the server?

default-artifact-root
serve-artifacts
artifacts-only
artifacts-destination

In [30]:
! mlflow ui --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts

2022/05/31 07:22:08 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/05/31 07:22:08 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

The argument `default-artifact-root` needs to be passed to properly configure the server.

# Q5. Tune the hyperparameters of the model
Now let's try to reduce the validation error by tuning the hyperparameters of the random forest regressor using hyperopt.

We have prepared the script hpo.py for this exercise.

Your task is to modify the script hpo.py and make sure that the validation RMSE is logged to MLflow for each run of the hyperparameter optimization (you will need to add a few lines of code to the objective function) and run the script without passing any parameters.

After that, open the MLflow UI and explore the runs from the experiment called random-forest-hyperopt to answer the question below.

Important: don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

the list of hyperparameters that are passed to the objective function during the optimization.
the RMSE obtained on the validation set (February 2021 data).
What's the best validation RMSE that you got?

6.128
6.628
7.128
7.628


In [35]:
! python3 hpo.py --data_path ./data

INFO: 'random-forest-hyperopt' does not exist. Creating a new experiment
100%|█████████| 50/50 [08:40<00:00, 10.41s/trial, best loss: 6.6284257482044735]


Best loss: 6.6284

# Q6. Promote the best model to the model registry
The results from the hyperparameter optimization are quite good so we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called register_model.py, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2021 data) and save the results to a new experiment called random-forest-best-models.

Your task is to update the script register_model.py so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method search_runs from the MlflowClient to get the model with the lowest RMSE. Tip 2: to register the model you can use the method mlflow.register_model and you will need to pass the right model_uri in the form of a string that looks like this: "runs:/<RUN_ID>/model", and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

6.1
6.55
7.93
15.1


In [37]:
! python3 register_model.py --data_path ./data

2022/05/31 07:56:14 WARNING mlflow.sklearn: Failed to infer model signature: Expected one of (pandas.DataFrame, numpy array, dictionary of (name -> numpy.ndarray), pyspark.sql.DataFrame) but got '<class 'scipy.sparse.csr.csr_matrix'>'
2022/05/31 07:56:34 WARNING mlflow.sklearn: Failed to infer model signature: Expected one of (pandas.DataFrame, numpy array, dictionary of (name -> numpy.ndarray), pyspark.sql.DataFrame) but got '<class 'scipy.sparse.csr.csr_matrix'>'
2022/05/31 07:57:01 WARNING mlflow.sklearn: Failed to infer model signature: Expected one of (pandas.DataFrame, numpy array, dictionary of (name -> numpy.ndarray), pyspark.sql.DataFrame) but got '<class 'scipy.sparse.csr.csr_matrix'>'
2022/05/31 07:57:31 WARNING mlflow.sklearn: Failed to infer model signature: Expected one of (pandas.DataFrame, numpy array, dictionary of (name -> numpy.ndarray), pyspark.sql.DataFrame) but got '<class 'scipy.sparse.csr.csr_matrix'>'
2022/05/31 07:57:50 WARNING mlflow.sklearn: Failed to infer 

The minimum RMSE ontraining set is 6.548.